# **Word Embeddings e Geração de Texto** 

Para essa atividde você irá usar documentos jurídicos coletados da plataforma Jusbrasil. Mais especificamente, preparamos um conjunto de textos inteiro teor de jurisprudências. Os dados já vem com algum processamento, mas sugerimos aplicar ainda outros filtros como remoção de acentos, cedilhas e caratecres especiais. Os dados se encontram nesse link.

## **Descrição**

Nesta atividade você irá exercitar as técnicas de Deep Learning que aprendeu na disciplina na seguintes tarefas.

## **Word Embeddings**

Treine um modelo skip-gram nesses dados e faça as seguintes consultas. Consulte o modelo sobre as palavras mais similares a: juiz e crime. Rode as seguintes analias:

    promotora está para juiz assim como promotor está para o que?
    acusado está para réu assim como testemunha está para o que?
    Proponha mais três analogias como essas para demonstrar que os emebeddings foram bem treinados.

## **Geração de Texto**

Nessa atividade, você vai treinar um modelo LSTM (similar ao que vimos nas aulas) para geração de texto jurídico. Se a memória do Colab "reclamar", use uma amostra menor dos dados.

Experimente incluir uma camada de embedding ao modelo (esse [link](https://keras.io/guides/working_with_rnns/) pode ajudar).
Experimente vários valores de temperatura (similar ao que vimos na aula).
Analise as saídas dos modelos quanto coerência e gramática.

# **Imports**

In [1]:
import numpy as np
import pandas as pd
import gensim
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from google.colab import drive
from sklearn.utils import shuffle
from tensorflow.keras.optimizers.experimental import RMSprop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Load Data**

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_all = pd.read_csv("/content/drive/MyDrive/teor_inteiro_jusbrasil.csv")
df_all = df_all[['text']]
df_all

,text
0,recurso extraordinário com agravo 1.126.769 ri...
1,a c ó r d ã o (3 turma) gmmgd/ls/dsc agravo. a...
2,poder judiciário tribunal de justiça do estad...
3,ed nº 70055698377 (nº cnj: 029446471.2013.8.2...
4,poder judiciário tribunal de justiça estado ...
...,...
3324,íntegra do acórdão ocultar acórdão atenção: ...
3325,poder judiciário tribunal de justiça do estad...
3326,ementa: remessa necessária condenação de prov...
3327,poder judiciário tribunal de justiça 1 vicep...


In [ ]:
def select_by_word(words, column, df):
  dic = {column: []}
  for text in df[column]:
    for w in words:
      if (w in text):
        dic[column].append(text)
        break
  return pd.DataFrame(dic)


#words = ["promotora", "juiz", "promotor", "acusado", "réu", "testemunha", "poder", "judiciário", "tribunal", "tempo", "trabalho", "fgts", "banco", "dinheiro", "juíza"]
#df = select_by_word(words, 'text', df_all)
df = df_all.sample(1000)
df

,text
2830,a c ó r d ã o (ac. 3 turma) gmalb/as/maf/ab/ri...
2013,poder judiciário do estado do rio de janeiro ...
1494,pgzpzwxku2v0igrpc2fibgvkpsjkaxnhymxlzci+pd94bw...
1453,poder judiciário tribunal regional do trabalho...
602,a c ó r d ã o (3 turma) gmmgd/rat/lnc/dsc agra...
...,...
1355,poder judiciário justiça do trabalho tribuna...
747,a c ó r d ã o (sdi1) gmacc/mda/jk/m embargos ...
1888,poder judiciário tribunal de justiça do estad...
910,poder judiciário justiça do trabalho tribunal ...


# **Pre-Processing Text**

In [ ]:
import unicodedata

def pre_processing_text(text):

  # Remove Accentuation
  text = ''.join(ch for ch in unicodedata.normalize('NFKD', text) 
    if not unicodedata.combining(ch))
  return text.lower()

df['text'] = df['text'].apply(lambda x: pre_processing_text(x))

In [ ]:
df.sample(5)

,text
1565,poder judiciario federal justica do trabalho ...
2584,poder judici&aacute;rio justi&ccedil;a do tr...
527,estado de rondonia poder judiciario tribunal d...
1639,(processo eletronico) ihmn no 70083356444 (no...
2863,ementa: apelacao civel mandado de seguranca ...


# **Word Embeddings**

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
text = ' '.join([value for index, value in df['text'].items()])

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer('\w+')
input_data = [tokenizer.tokenize(sent) for sent in sent_tokenize(text, "portuguese")]

In [ ]:
num_features = 300
min_word_count = 3
num_workers = 2
window_size = 6
subsampling = 1e-3
model = Word2Vec(input_data, workers=num_workers, size=num_features, min_count=min_word_count, window=window_size, sample=subsampling)

In [ ]:
model.most_similar(positive=["juiz", "crime"], topn=5)

<ipython-input-57-c92f68687b0e>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=["juiz", "crime"], topn=5)


[('furto', 0.6484065651893616),
 ('acusado', 0.6471056938171387),
 ('comportamento', 0.6449525356292725),
 ('cidadao', 0.6342822313308716),
 ('cometimento', 0.6155557632446289)]

In [ ]:
model.similar_by_word("juiz", topn=5)

<ipython-input-58-40bcf1e9fb90>:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word("juiz", topn=5)


[('julgador', 0.7052947282791138),
 ('juiza', 0.7025941610336304),
 ('desembargador', 0.670617938041687),
 ('magistrado', 0.6480852365493774),
 ('substituto', 0.6459724307060242)]

In [ ]:
model.similar_by_word("crime", topn=5)

<ipython-input-59-d5e7727001d4>:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  model.similar_by_word("crime", topn=5)


[('delito', 0.881077229976654),
 ('furto', 0.8170905113220215),
 ('trafico', 0.7977221608161926),
 ('cometimento', 0.7780550122261047),
 ('roubo', 0.7396363019943237)]

In [ ]:
def analogous_word(word_1, word_2, word_3, model):
  ''' The function accepts a triad of words, word_1, word_2, word_3 and returns word_4 such that word_1:word_2::word_3:word_4 '''
  # converting each word to its lowercase
  word_1,word_2,word_3 = word_1.lower(),word_2.lower(),word_3.lower()

  # Similarity between |word_2-word_1| = |word_4-word_3| should be maximum
  maximum_similarity = -99999

  word_4 = None

  words = model.wv.vocab

  va,vb,vc = model.wv.__getitem__(word_1), model.wv.__getitem__(word_2), model.wv.__getitem__(word_3)

  # to find word_4 such that similarity
  # (|word_2 - word_1|, |word_4 - word_3|) should be maximum

  for i in words:
    if i in [word_1,word_2,word_3]:
      continue
    wvec = model.wv.__getitem__(i)
    similarity = cosine_similarity([wvec-vc])
    if similarity > maximum_similarity:
      maximum_similarity = similarity
      word_4 = i
  return word_4

In [ ]:
triad = ("promotora", "juiz", "promotor")
analogous_word(*triad, model)

'alheios'

In [ ]:
triad = ("acusado", "reu", "testemunha")
analogous_word(*triad, model)

'argui'

In [ ]:
triad = ("poder", "judiciario", "tribunal")
analogous_word(*triad, model)

'conta'

In [ ]:
triad = ("crime", "acusado", "dinheiro")
analogous_word(*triad, model)

'executada'

In [ ]:
triad = ("banco", "dinheiro", "juiza")
analogous_word(*triad, model)

'441'

# **Text Generation**

In [ ]:
text = ""
for txt in df.sample(100).text:
  text += txt + " "

In [ ]:
chars = sorted(list(set(text)))

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
'corpus length: {} total chars: {}'.format(len(text), len(chars))

'corpus length: 1390750 total chars: 64'

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 463570


In [ ]:
sentences[2]

'a do acordao ocultar acordao   atencao: '

In [ ]:
next_chars[2]

'o'

In [ ]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

<ipython-input-72-2335da19742b>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-72-2335da19742b>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars),activation='softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
Total params: 107,072
Trainable params: 107,072
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 30
batch_size = 128
model.fit(X, y, batch_size=batch_size, epochs=epochs)

Epoch 1/30
3622/3622 [==============================] - 29s 6ms/step - loss: 2.1871
Epoch 2/30
3622/3622 [==============================] - 22s 6ms/step - loss: 1.7810
Epoch 3/30
3622/3622 [==============================] - 23s 6ms/step - loss: 1.5985
Epoch 4/30
3622/3622 [==============================] - 24s 7ms/step - loss: 1.4909
Epoch 5/30
3622/3622 [==============================] - 21s 6ms/step - loss: 1.4190
Epoch 6/30
3622/3622 [==============================] - 21s 6ms/step - loss: 1.3684
Epoch 7/30
3622/3622 [==============================] - 21s 6ms/step - loss: 1.3290
Epoch 8/30
3622/3622 [==============================] - 23s 6ms/step - loss: 1.2984
Epoch 9/30
3622/3622 [==============================] - 21s 6ms/step - loss: 1.2728
Epoch 10/30
3622/3622 [==============================] - 22s 6ms/step - loss: 1.2513
Epoch 11/30
3622/3622 [==============================] - 21s 6ms/step - loss: 1.2328
Epoch 12/30
3622/3622 [==============================] - 22s 6ms/step - lo

In [ ]:
import random

def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import sys
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0]:
  print()
  print('----- diversity:', diversity)
  generated = ''
  sentence = text[start_index: start_index + maxlen]
  generated += sentence
  print('----- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)
  for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x[0, t, char_indices[char]] = 1.
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    generated += next_char
    sentence = sentence[1:] + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()


----- diversity: 0.2
----- Generating with seed: "pelacao, foi aplicada a tecnica de julga"
pelacao, foi aplicada a tecnica de julgamento do recurso interposta pela autora a sentenca e de contrarrazoes ao agravo de instrumento pelo recurso de autoras de acordo com a construcao de prova manifestar a concessao de prova ora sua despeitos de forma consignada a prestacao do artigo 1o, inciso iii, da lei no 11.813/2015 e a apelacao civel no anca da contagem da sentenca no processo em que se existindo a ser alegacoes de atendes a aca

----- diversity: 0.5
----- Generating with seed: "pelacao, foi aplicada a tecnica de julga"
pelacao, foi aplicada a tecnica de julgamento e a segurada e nao se concedida por ajuizada pela sua autora a concessao do pelo dever a integrante do entendimento de aposentadoria por idade habitacao do estado de segunda secao de execucao por falta de possivel, confirmou que o pagamento do processo administrativo para o bem deve ser processada pela sua legisima desembarga